In [ ]:
import pandas as pd
df = pd.read_csv("DATASET 1.csv")

In [ ]:
# Count the number of rows per category
category_counts = df['Category'].value_counts()

# Keep only categories with at least 30 rows
valid_categories = category_counts[category_counts >= 30].index
filtered_df = df[df['Category'].isin(valid_categories)]

# For categories with more than 100 rows, keep only the first 100
limited_df = filtered_df.groupby('Category').head(100)

# Save the filtered result
limited_df.to_csv("raw_dataset.csv", index=False)


In [ ]:
print("Final Balanced Dataset Shape:", limited_df.shape)
print("Categories and Counts:\n", limited_df['Category'].value_counts())


Final Balanced Dataset Shape: (3627, 2)
Categories and Counts:
 Category
Accountant                 100
Advocate                   100
Arts                       100
Banking                    100
Aviation                   100
Construction               100
Consultant                 100
Chef                       100
Business Development       100
Information Technology     100
Healthcare                 100
Database Administrator     100
Designer                   100
Education                  100
Engineering                100
Web Developer              100
Sales                      100
HR                         100
Fitness                    100
Finance                    100
Engineering                100
Security Analyst           100
Teacher                    100
Information Technology     100
Network Administrator      100
Project manager            100
Public Relations           100
Python Developer           100
Apparel                     97
Digital Media               

In [ ]:
df = limited_df

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# === Step 1: Initialize TF-IDF Vectorizer ===
tfidf_vectorizer = TfidfVectorizer(
    max_features=1200,      # Top 1200 most important words/phrases
    ngram_range=(1, 2),     # Unigrams + Bigrams
    min_df=5,               # Appear in at least 5 resumes
    max_df=0.8              # Appear in at most 80% of resumes
)

# === Step 2: Fit and Transform on balanced_df ===
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Resume'])

# === Step 3: Confirm Shape ===
print("TF-IDF Matrix Shape", tfidf_matrix.shape)


TF-IDF Matrix Shape (3627, 1200)


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def extract_structural_features(text_series):
    features = []

    for text in text_series:
        tokens = text.split()
        num_words = len(tokens)
        num_chars = sum(len(word) for word in tokens)
        avg_word_length = num_chars / num_words if num_words else 0

        num_digits = sum(char.isdigit() for char in text)
        num_uppercase_words = sum(word.isupper() for word in tokens)
        num_unique_words = len(set(tokens))
        num_stopwords = sum(1 for word in tokens if word.lower() in ENGLISH_STOP_WORDS)

        features.append([
            num_words,
            avg_word_length,
            num_digits,
            num_uppercase_words,
            num_unique_words,
            num_stopwords
        ])

    return np.array(features)

# === Extract features from balanced_df ===
structural_features = extract_structural_features(df['Resume'])

# === Wrap in a DataFrame ===
import pandas as pd
# Rename the structural features for balanced dataset
structural_df= pd.DataFrame(
    structural_features,
    columns=[
        'resume_length',
        'avg_word_length',
        'num_digits',
        'num_uppercase_words',
        'num_unique_words',
        'num_stopwords'
    ]
)


# === Check output ===
print(" Structural Feature Matrix Shape:", structural_df.shape)
print(" Sample Structural Features:\n", structural_df.head())


 Structural Feature Matrix Shape: (3627, 6)
 Sample Structural Features:
    resume_length  avg_word_length  num_digits  num_uppercase_words  \
0          747.0         6.402945        42.0                 26.0   
1         1224.0         6.159314       106.0                 31.0   
2         1049.0         5.924690        84.0                 23.0   
3          659.0         6.303490        65.0                 25.0   
4          675.0         6.082963       141.0                 32.0   

   num_unique_words  num_stopwords  
0             401.0          173.0  
1             673.0          329.0  
2             485.0          290.0  
3             372.0          127.0  
4             390.0          119.0  


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load a lightweight, fast BERT model
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode the resumes
bert_embeddings = bert_model.encode(df['Resume'].tolist(), show_progress_bar=True)

# Convert to dense array (no need for avg_glove_vector anymore)
semantic_vectors_balanced = np.array(bert_embeddings)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/114 [00:00<?, ?it/s]

In [ ]:
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix

# === 1. Scale Structural Features ===
scaler_struct = StandardScaler()
struct_scaled_balanced = scaler_struct.fit_transform(structural_df)

# === 2. Scale Semantic Features ===
scaler_sem = StandardScaler()
semantic_scaled_balanced = scaler_sem.fit_transform(semantic_vectors_balanced)

# === 3. Convert to Sparse Matrices ===
struct_sparse_balanced = csr_matrix(struct_scaled_balanced)
semantic_sparse_balanced = csr_matrix(semantic_scaled_balanced)

# === 4. Stack TF-IDF + Structural + Semantic ===
X_np = hstack([
    tfidf_matrix,
    struct_sparse_balanced,
    semantic_sparse_balanced
])

# Final shape check
print("Final Feature Matrix Shape:", X_np.shape)



Final Feature Matrix Shape: (3627, 1590)


In [ ]:
# Convert sparse matrix to dense (if it's not too large)
X_dense = X_np.toarray()  # Use only if the data is small/medium size!


df['embedding'] = [vec.tolist() for vec in X_dense]

# Check the first embedding vector
print(df["embedding"][0])

[0.0, 0.09846995958632149, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.019275457879396874, 0.03537841003054677, 0.0, 0.0, 0.0, 0.02805789750157089, 0.0, 0.0, 0.06207122899920598, 0.0, 0.0, 0.0, 0.0, 0.02568953058041774, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018269958849760733, 0.0, 0.0, 0.01782288634969645, 0.0, 0.02322732385926475, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.024281147525339178, 0.0, 0.0, 0.0, 0.0, 0.024809575965438556, 0.1561349275479466, 0.409360593521653, 0.07158602640335288, 0.05794284807787483, 0.027545409637908542, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03565510313425503, 0.0, 0.0, 0.0, 0.020571937502917713, 0.0, 0.0, 0.0, 0.0, 0.0, 0.027824767089117096, 0.0, 0.0, 0.0, 0.0, 0.03902838988074978, 0.0, 0.0, 0.0, 0.0, 0.030274140366317225, 0.030885127288150765, 0.0925748579823332, 0.059180022362145486, 0.0, 0.05629818979577493, 0.0, 0.05561413334068517, 0.0274427146478899, 0.0, 0.

<ipython-input-22-334bfa650a62>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embedding'] = [vec.tolist() for vec in X_dense]


In [ ]:
# Save the filtered result
df.to_csv("embedded_dataset.csv", index=False)

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from collections import Counter

# === Step 1: Apply LabelEncoder to target ===
label_encoder = LabelEncoder()
y_bal_encoded = label_encoder.fit_transform(df['Category'])

# === Step 2: Apply SMOTE on full balanced dataset ===
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_np, y_bal_encoded)

# === Optional: Check new class distribution ===
print("Class Distribution After SMOTE:", Counter(y_smote))

# === Step 3: Train-test split on the new oversampled data ===
X_train, X_temp, y_train, y_temp = train_test_split(
    X_smote, y_smote, test_size=0.3, stratify=y_smote, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# === Step 4: Convert back to original labels (if needed for final prediction/report) ===
y_train_labels = label_encoder.inverse_transform(y_train)
y_val_labels = label_encoder.inverse_transform(y_val)


Class Distribution After SMOTE: Counter({np.int64(0): 100, np.int64(1): 100, np.int64(2): 100, np.int64(3): 100, np.int64(4): 100, np.int64(5): 100, np.int64(6): 100, np.int64(7): 100, np.int64(8): 100, np.int64(9): 100, np.int64(10): 100, np.int64(11): 100, np.int64(12): 100, np.int64(13): 100, np.int64(14): 100, np.int64(15): 100, np.int64(16): 100, np.int64(17): 100, np.int64(18): 100, np.int64(19): 100, np.int64(21): 100, np.int64(23): 100, np.int64(24): 100, np.int64(25): 100, np.int64(26): 100, np.int64(27): 100, np.int64(28): 100, np.int64(29): 100, np.int64(31): 100, np.int64(32): 100, np.int64(33): 100, np.int64(34): 100, np.int64(35): 100, np.int64(36): 100, np.int64(37): 100, np.int64(38): 100, np.int64(39): 100, np.int64(40): 100, np.int64(41): 100, np.int64(42): 100, np.int64(43): 100, np.int64(44): 100, np.int64(30): 100, np.int64(20): 100, np.int64(22): 100})


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

# === Train XGBoost on SMOTE-balanced data ===
xgb_clf = XGBClassifier(
    objective='multi:softprob',
    num_class=len(label_encoder.classes_),
    eval_metric='mlogloss',
    learning_rate=0.1,
    max_depth=6,
    n_estimators=200,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_clf.fit(X_train, y_train)

# === Predict on Validation Set ===
y_val_pred = xgb_clf.predict(X_val)
y_val_pred_labels = label_encoder.inverse_transform(y_val_pred)

# === Evaluation ===
print("Validation Accuracy:", accuracy_score(y_val_labels, y_val_pred_labels))
print("\nClassification Report (Validation):")
print(classification_report(y_val_labels, y_val_pred_labels))


✅ Validation Accuracy: 0.7644444444444445

📊 Classification Report (Validation):
                         precision    recall  f1-score   support

            Accountant        0.79      1.00      0.88        15
              Advocate        0.58      0.47      0.52        15
           Agriculture        0.93      0.87      0.90        15
               Apparel        0.64      0.47      0.54        15
                  Arts        0.82      0.60      0.69        15
            Automobile        0.77      0.67      0.71        15
              Aviation        0.92      0.73      0.81        15
               Banking        0.64      0.60      0.62        15
            Blockchain        1.00      1.00      1.00        15
  Business Development        0.86      0.80      0.83        15
                  Chef        0.88      1.00      0.94        15
          Construction        0.74      0.93      0.82        15
            Consultant        0.64      0.60      0.62        15
        

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# === Predict on Test Set ===
y_test_pred_enc = xgb_clf.predict(X_test)

# === Convert Encoded Labels Back to Original Class Names ===
y_test_pred = label_encoder.inverse_transform(y_test_pred_enc)
y_test_true = label_encoder.inverse_transform(y_test)

# === Evaluate Test Set Performance ===
print("✅ Test Accuracy:", accuracy_score(y_test_true, y_test_pred))
print("\n📊 Classification Report (Test):")
print(classification_report(y_test_true, y_test_pred))


✅ Test Accuracy: 0.7540740740740741

📊 Classification Report (Test):
                         precision    recall  f1-score   support

            Accountant        0.88      0.93      0.90        15
              Advocate        0.67      0.40      0.50        15
           Agriculture        0.68      0.87      0.76        15
               Apparel        0.83      0.33      0.48        15
                  Arts        0.75      0.60      0.67        15
            Automobile        0.85      0.73      0.79        15
              Aviation        0.93      0.93      0.93        15
               Banking        0.67      0.67      0.67        15
            Blockchain        0.94      1.00      0.97        15
  Business Development        0.80      0.80      0.80        15
                  Chef        0.71      0.80      0.75        15
          Construction        0.68      1.00      0.81        15
            Consultant        0.91      0.67      0.77        15
          Data Scien